In [1]:
import bz2
import json
import requests as rq
import time

In [2]:
dump = "/datasets_1/sagarj/reddit_all_2017/RC_2017-02.bz2"

In [3]:
openedDump = bz2.BZ2File(dump, "r")

In [4]:
count = 0
for line in openedDump:
    if count > 10:
        break
    print repr(line)
    count+=1

'{"author_flair_text":null,"id":"dd61ge9","edited":false,"subreddit_id":"t5_2zgmb","stickied":false,"retrieved_on":1488453514,"author_flair_css_class":null,"created_utc":1485907200,"controversiality":0,"link_id":"t3_5qxrvb","parent_id":"t1_dd3vflb","subreddit":"AshevilleMusic","author":"Durgroth","distinguished":null,"body":"I have no idea what these places are other than the Mothlight, but I\'ll definitely check them out!","gilded":0,"score":1}\n'
'{"edited":false,"stickied":false,"retrieved_on":1488453514,"subreddit_id":"t5_2qh33","id":"dd61gea","author_flair_text":null,"distinguished":null,"author":"SlothOfDoom","parent_id":"t1_dd5zw81","subreddit":"funny","score":16,"gilded":0,"body":"She was also one of the very first Federation ships built specifically as a warship (although officially it is an \'escort\' so the Federation can still look touchy-feely)","created_utc":1485907200,"author_flair_css_class":null,"link_id":"t3_5r9d4o","controversiality":0}\n'
'{"edited":false,"stickied"

In [15]:
def getJson(url):
    session_headers = {
    'Host': 'www.reddit.com',
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    #'Referer': 'http://localhost:9000/notebooks/Reddit/Notebooks/SuicideWatchSubreddit.ipynb'
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-GB,en-US;q=0.8,en;q=0.6',
    'Cookie': "loid=00000000000gslzvdi.2.1507591930964.Z0FBQUFBQlozQWI3M243MTdFMUgzNWtQNzNVbzg5Z3NXYmxpLUdRTWVXdEpWQzBfbkxBSVNNVVF3dE1zQlN5cVMzeGtxZXlmT0JQY2JvTld2NWlVeGlGeFpkNnBYM2ZIYlNNR2xMWGFFS3hieDZpWEJ5OVlyTng0MUFZZjNYdjQ0ZURFNTFlSmlVRGw; edgebucket=T8nkTF0z7WPnNzlE8B; __gads=ID=ab9c1b0fcb7eaefc:T=1507591936:S=ALNI_Mao2tvEoyJzEDEuxxhfZBlOQHXlsA; eu_cookie_v2=3; _ga=GA1.2.1088810883.1507591931; __utma=55650728.1088810883.1507591931.1507912304.1507912553.14; __utmz=55650728.1507912553.14.11.utmcsr=reddit|utmccn=(not%20set)|utmcmd=hot|utmcct=comments; aa=1; _recentclicks2=t3_7cfzsx%2Ct3_7cfw08%2Ct3_7cdofu%2Ct3_7caog1%2Ct3_7cdvud; _recent_srs=t5_2qpzs%2Ct5_2qh1q%2Ct5_2qh49%2Ct5_2rtff%2Ct5_36buk%2Ct5_2r7yd%2Ct5_2rks3%2Ct5_3j2jr%2Ct5_2sumw%2Ct5_2xp2o; pc=r3; session_tracker=tPgx9U9yPt99jYRDqC.0.1510585492994.Z0FBQUFBQmFDYlNWcWZHclI5WWxDWW9kQ0dURk9FRU9BZjFJTURWZElzaldRbkROaEJUOWE1czRQUWxPUzUwOFFwY3R3ckI5VkNDUlhJc01NVEtEeHVWd25CdUpXWEI1X012RkJaS3cyNXdNbk5vOE9DSXp5ZlhuTzF3a2pxT2dLZ3laMDhtQlhib3k; initref=localhost"
    }

    client = rq.session()
    resp = client.get(url , headers=session_headers)
    if resp.status_code == 200:
        json_data = resp.json()
        return json_data
    else:
        print "Failed to get page"
        return None
def getSubredditId(json):
    if type(json) == dict:
        sub_id = json['data']['children'][0]['data']['subreddit_id']
        return sub_id
    else: 
        print "Invalid Format"
        return None

def makeSubUrl(name):
    return "https://www.reddit.com/r/" + name + "/.json"

def getSubIds(RedditList):
    subIds = {}
    for name in RedditList:
        json = getJson(makeSubUrl(name))
        subIds[name] = getSubredditId(json)
    return subIds

def filterDumpbySub(filePath , subID_dict, limit=None):
    import json
    import bz2
    
    openedDump = bz2.BZ2File(filePath, "r")
    count = 0
    returnData = dict()
    print "=============Parsing============"
    print subID_dict
    st = time.time()
    for line in openedDump:
        lineData = json.loads(line.strip())
        if (limit!=None and count > limit):
            return returnData
        if lineData['subreddit'] in subID_dict:
            if lineData['subreddit'] not in returnData:
                returnData[lineData['subreddit']] = []
            postId = lineData['link_id'].strip().split('_')[1]
            
            if postId not in returnData[lineData['subreddit']]:
                returnData[lineData['subreddit']].append(postId)
        if count%100000 == 0:
            end = time.time()
            print "Done parsing %d posts in %d seconds"%(count,(end-st))
            st = time.time()
        count+=1
    return returnData

In [47]:
subreddits = ["mentalhealth","cripplingalcoholism" , "disorder" , "Health" , "addiction" , "ADHD", "depression",'TheDonald' , 'SuicideWatch'] 
# subreddits = ['depression','bipolar','suicidewatch' ]

In [48]:
subID_dict = getSubIds(subreddits)

In [49]:
subID_dict

{'ADHD': u't5_2qnwb',
 'Health': u't5_2qh9z',
 'SuicideWatch': u't5_2qpzs',
 'TheDonald': u't5_2ve48',
 'addiction': u't5_2qwvc',
 'cripplingalcoholism': u't5_2s68b',
 'depression': u't5_2qqqf',
 'disorder': u't5_2qrlr',
 'mentalhealth': u't5_2qirg'}

In [ ]:
postData = filterDumpbySub(dump,subID_dict)

=============Parsing============
{'mentalhealth': u't5_2qirg', 'SuicideWatch': u't5_2qpzs', 'cripplingalcoholism': u't5_2s68b', 'Health': u't5_2qh9z', 'ADHD': u't5_2qnwb', 'TheDonald': u't5_2ve48', 'addiction': u't5_2qwvc', 'disorder': u't5_2qrlr', 'depression': u't5_2qqqf'}
Done parsing 0 posts in 0 seconds
Done parsing 100000 posts in 3 seconds
Done parsing 200000 posts in 3 seconds
Done parsing 300000 posts in 3 seconds
Done parsing 400000 posts in 2 seconds


In [ ]:
postData.keys()

In [ ]:
len(postData['bipolar'])

In [ ]:
import pickle as pkl
with open("/datasets_1/sagarj/IoPPN_collab/MentalHealth_2017_posts.pkl",'w') as f:
    pkl.dump(postData,f)

In [ ]:
len(list(set(postData['depression'])))

In [ ]:
# import pickle as pkl
# with open("ChangemyView2017.pkl",'w') as f:
#     pkl.dump(postData['changemyview'],f)

In [ ]:
# with open("Filtered_posts_2017.json",'w') as f:
#     json.dump(postData,f)